In [18]:
from zipfile import ZipFile
import os

base_folder = "./data"
file_name = os.path.join(base_folder, 'brains.zip')
data_folder = os.path.join(base_folder, 'brains')

with ZipFile(file_name, 'r') as zip:
    zip.extractall(data_folder)


In [40]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
import os

folder_path = os.path.join(data_folder, 'Brain Tumor Data Set/Brain Tumor Data Set')

train_ds = image_dataset_from_directory(
    folder_path,
  validation_split=0.2,
  seed=123,
    subset='training',
  image_size=(512, 512),
  batch_size=30)

val_ds = image_dataset_from_directory(
    folder_path,
  validation_split=0.2,
  seed=123,
  image_size=(512, 512),
    subset='validation',
  batch_size=30)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(1)
val_ds = val_ds.prefetch(1)

Found 4514 files belonging to 2 classes.
Using 3612 files for training.
Found 4514 files belonging to 2 classes.
Using 902 files for validation.


In [45]:
from keras import models
from keras import layers

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),
    loss='binary_crossentropy',
    metrics=['acc']
)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 510, 510, 32)      896       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 253, 253, 64)      18496     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 124, 124, 128)     73856     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 60, 60, 128)      

In [46]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

filename = 'Brain.h5'

checkpoint = ModelCheckpoint(filename,             
                             monitor='val_loss',   
                             verbose=1,            
                             save_best_only=True,  
                             mode='auto'           
                            )

earlystopping = EarlyStopping(monitor='val_loss',  
                              patience=10,         
                             )


In [ ]:
history =  model.fit(
    train_ds,
    epochs=60,
    validation_data=val_ds,
    callbacks=[checkpoint, earlystopping],
)

Epoch 1/60
121/121 [==============================] - 25s 202ms/step - loss: 0.8363 - acc: 0.7074 - val_loss: 0.5290 - val_acc: 0.7727

Epoch 00001: val_loss improved from inf to 0.52898, saving model to Brain.h5
Epoch 2/60
121/121 [==============================] - 25s 202ms/step - loss: 0.4065 - acc: 0.8403 - val_loss: 0.3820 - val_acc: 0.8326

Epoch 00002: val_loss improved from 0.52898 to 0.38203, saving model to Brain.h5
Epoch 3/60
121/121 [==============================] - 25s 202ms/step - loss: 0.2556 - acc: 0.8995 - val_loss: 0.2862 - val_acc: 0.8902

Epoch 00003: val_loss improved from 0.38203 to 0.28615, saving model to Brain.h5
Epoch 4/60
121/121 [==============================] - 25s 203ms/step - loss: 0.1903 - acc: 0.9277 - val_loss: 0.2282 - val_acc: 0.9257

Epoch 00004: val_loss improved from 0.28615 to 0.22825, saving model to Brain.h5
Epoch 5/60
121/121 [==============================] - 25s 202ms/step - loss: 0.1271 - acc: 0.9607 - val_loss: 0.1932 - val_acc: 0.9368

